<a href="https://colab.research.google.com/github/0ShNa0/DatasetGE/blob/main/GE_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import json
from sys import modules
compile2=[]
with open('output_data.json') as json_file:
  data=json.load(json_file)
  for d in data:
    for block in d['ai-generated']:
      if(block['category']==2):
        sent=block['sentence'].lstrip()
        if(sent[0]>='0' and sent[0]<='9' and sent[1]=='.'):
          compile2.append(sent[2:])
        else:
          compile2.append(sent)

In [16]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")


ner= spacy.load('en_core_web_lg')


def spacy_large_ner(document):
  return [ent.text.strip()for ent in ner(document).ents if ent.label_ in ['GPE', 'LOC']]



✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [17]:
process_data={}
for sent in compile2:
  ents=spacy_large_ner(sent)
  process_data[sent]=ents


In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=10,
    num_beam_groups=10,
    num_return_sequences=10,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [19]:
process_data2={}
for keys,_ in process_data.items():
  process_data2[keys]=paraphrase(keys)
  print(process_data2[keys])

['They were prompted to wear bright outfits and bring bells, flowers, and other objects as props.', 'Bright clothing and accessories, such as flowers and bells, were given to them for decoration.', 'The group was urged to wear costumes with vibrant colors and bring bells, flowers, and other objects as props.', 'To dress in bright colors, they were urged to bring bells, flowers, and other props for decoration.', 'Dressed in bright colors, they were urged to use bells and flowers and other objects as props.', 'Bringing bells, flowers and other props was also encouraged with the intention to dress in bright colours.', 'They were also asked for bright costumes and to bring bells, flowers and other props as decorations.', 'Bells, flowers and other objects were brought for them to prop their outfits while they wore bright colours.', 'The individuals were motivated to put on vibrant outfits and brought along ornamental items like bells, flowers, and other objects for decoration.', 'Their atti

In [20]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

def check_entailment(query, text):
  tokenizer = RobertaTokenizer.from_pretrained('roberta-large-mnli')
  model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli')
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  inputs = tokenizer.encode_plus(query, text, add_special_tokens=True, padding='longest', truncation=True, return_tensors='pt')
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    entailment_score = probabilities[:, 1].item()

  return entailment_score


In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
scorelist=dict()
for key,rephrase in process_data2.items():
  scorelist[key]=[check_entailment(key,sentence) for sentence in rephrase]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT

In [23]:
print(scorelist)

{'They were encouraged to dress in bright colors and bring bells, flowers, and other objects to use as props.': [0.004658909980207682, 0.008163795806467533, 0.005693253595381975, 0.024068953469395638, 0.004105434287339449, 0.005488673225045204, 0.01660780794918537, 0.007911497727036476, 0.00582655007019639, 0.0054245502687990665], 'Buttigieg’s performance in Ohio was not his strongest, as he was unable to make a significant dent in the state’s primary race.': [0.3656000792980194, 0.22503939270973206, 0.004762627650052309, 0.022490818053483963, 0.9964646100997925, 0.9913470149040222, 0.8272374272346497, 0.7613617181777954, 0.054215896874666214, 0.17517852783203125], 'While his performance in Ohio was not his best, he is still a formidable contender for the presidential nomination.': [0.007039859890937805, 0.007191616576164961, 0.021638691425323486, 0.02727096900343895, 0.015295998193323612, 0.06653650850057602, 0.5477586388587952, 0.024994028732180595, 0.00914524495601654, 0.03455457463

In [28]:
csv_file_name = 'output2.csv'
import csv
# Writing the dictionary to a CSV file
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(scorelist.keys())

    # Write the data
    csv_writer.writerows(zip(*scorelist.values()))

In [29]:
csv_file_name = 'process_data2GE.csv'
import csv
# Writing the dictionary to a CSV file
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(process_data2.keys())

    # Write the data
    csv_writer.writerows(zip(*process_data2.values()))

In [30]:

from google.colab import files
files.download('output2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:

from google.colab import files
files.download('process_data2GE.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
import csv

def csv_to_dict(file_path):
    result_dict = {}
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            for key, value in row.items():
                result_dict.setdefault(key, []).append(value)
    return result_dict

# Example usage
csv_file_path = 'output2.csv'
result_dict = csv_to_dict(csv_file_path)

print(result_dict)


{'They were encouraged to dress in bright colors and bring bells, flowers, and other objects to use as props.': ['0.004658909980207682', '0.008163795806467533', '0.005693253595381975', '0.024068953469395638', '0.004105434287339449', '0.005488673225045204', '0.01660780794918537', '0.007911497727036476', '0.00582655007019639', '0.0054245502687990665'], 'Buttigieg’s performance in Ohio was not his strongest, as he was unable to make a significant dent in the state’s primary race.': ['0.3656000792980194', '0.22503939270973206', '0.004762627650052309', '0.022490818053483963', '0.9964646100997925', '0.9913470149040222', '0.8272374272346497', '0.7613617181777954', '0.054215896874666214', '0.17517852783203125'], 'While his performance in Ohio was not his best, he is still a formidable contender for the presidential nomination.': ['0.007039859890937805', '0.007191616576164961', '0.021638691425323486', '0.02727096900343895', '0.015295998193323612', '0.06653650850057602', '0.5477586388587952', '0

In [33]:
allscores=[]
for _,values in result_dict.items():
  for val in values:
    allscores.append(float(val))
print(allscores)

import numpy as np

sorted_scores = np.sort(allscores)


median = np.median(sorted_scores)

[0.004658909980207682, 0.008163795806467533, 0.005693253595381975, 0.024068953469395638, 0.004105434287339449, 0.005488673225045204, 0.01660780794918537, 0.007911497727036476, 0.00582655007019639, 0.0054245502687990665, 0.3656000792980194, 0.22503939270973206, 0.004762627650052309, 0.022490818053483963, 0.9964646100997925, 0.9913470149040222, 0.8272374272346497, 0.7613617181777954, 0.054215896874666214, 0.17517852783203125, 0.007039859890937805, 0.007191616576164961, 0.021638691425323486, 0.02727096900343895, 0.015295998193323612, 0.06653650850057602, 0.5477586388587952, 0.024994028732180595, 0.00914524495601654, 0.03455457463860512, 0.177032932639122, 0.026775451377034187, 0.07601773738861084, 0.04619418457150459, 0.010590305551886559, 0.022295350208878517, 0.047140203416347504, 0.02495817095041275, 0.0314486026763916, 0.019400088116526604, 0.006035690661519766, 0.0065572443418204784, 0.0066637443378567696, 0.005645048338919878, 0.005189380142837763, 0.011967512778937817, 0.3606163859

In [34]:
final_data=[]
for key,values1,values2 in zip(result_dict.keys(),process_data2.values(),result_dict.values()):
  for sent,score in zip(values1,values2):
    if(float(score)>=median and key):
      final_data.append([key,sent])

In [35]:

for data in final_data:
  print(data)

['They were encouraged to dress in bright colors and bring bells, flowers, and other objects to use as props.', 'Bright clothing and accessories, such as flowers and bells, were given to them for decoration.']
['They were encouraged to dress in bright colors and bring bells, flowers, and other objects to use as props.', 'To dress in bright colors, they were urged to bring bells, flowers, and other props for decoration.']
['They were encouraged to dress in bright colors and bring bells, flowers, and other objects to use as props.', 'They were also asked for bright costumes and to bring bells, flowers and other props as decorations.']
['Buttigieg’s performance in Ohio was not his strongest, as he was unable to make a significant dent in the state’s primary race.', 'Despite his efforts, Buttigieg failed to make a significant impact in Ohio during the primary.']
['Buttigieg’s performance in Ohio was not his strongest, as he was unable to make a significant dent in the state’s primary race.

In [40]:
import pandas as pd
df = pd.DataFrame((final_data), columns=["Column1", "Column2"])
print(df)
import csv

custom_header = ["Original","Paraphrases"]
df.columns = custom_header


df.to_csv("final_data2.csv", index=True)

                                               Column1  \
0    They were encouraged to dress in bright colors...   
1    They were encouraged to dress in bright colors...   
2    They were encouraged to dress in bright colors...   
3    Buttigieg’s performance in Ohio was not his st...   
4    Buttigieg’s performance in Ohio was not his st...   
..                                                 ...   
865  The two were buried together in one coffin, in...   
866  The two were buried together in one coffin, in...   
867  The two were buried together in one coffin, in...   
868  The two were buried together in one coffin, in...   
869  The two were buried together in one coffin, in...   

                                               Column2  
0    Bright clothing and accessories, such as flowe...  
1    To dress in bright colors, they were urged to ...  
2    They were also asked for bright costumes and t...  
3    Despite his efforts, Buttigieg failed to make ...  
4    However, Butt

In [41]:
print(df.shape)

(870, 2)


In [42]:

from google.colab import files
files.download('final_data2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>